# Power Analysis

In [82]:
import pandas as pd
import numpy as np
import statsmodels.stats.power as smp
from scipy import stats
import plotly
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"

In [28]:
from __future__ import division #Ensure division returns float
from numpy import mean, std # version >= 1.7.1 && <= 1.9.1
from math import sqrt
import sys


def cohen_d(x,y):
        return (mean(x) - mean(y)) / sqrt((std(x, ddof=1) ** 2 + std(y, ddof=1) ** 2) / 2.0)

In [22]:
power_analysis = smp.TTestIndPower()
power_analysis.solve_power(effect_size=0.5,power=0.8,alpha=0.05)

63.765611775409525

**The results of the power test tells us we need at least 64 samples to see an effect size of 0.5 at 95% confidence. But what does an effect size of 0.5 look like?**

In [92]:
# draw distributions
x1_mean = 2.2
x1_scale = 2.4
x2_mean = 4.1
x2_scale = 2.2
N=30

df = pd.DataFrame(
    {'x1':np.random.normal(loc=x1_mean, scale=x1_scale, size=N)
     ,'x2':np.random.normal(loc=x2_mean, scale=x2_scale, size=N)
    })

fig = go.Figure()
fig.add_trace(go.Histogram(x=df['x1'],name='x1 mean={:0.1f}, std={:0.1f}'.format(df['x1'].mean(),df['x1'].std())))
fig.add_vline(df['x1'].mean())
fig.add_trace(go.Histogram(x=df['x2'],name='x2 mean={:0.1f}, std={:0.1f}'.format(df['x2'].mean(),df['x2'].std())))
fig.add_vline(df['x2'].mean())

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

# effect size in title
d = cohen_d(df['x1'],df['x2'])
p = power_analysis.solve_power(effect_size=d,power=0.8,alpha=0.05)
t,p_value = stats.ttest_ind(df['x1'].values,df['x2'].values)
title = "Normal Distributions N={:0.0f} Effect Size = {:0.1f} (Power @ N={:0.1f}) t={:0.1f} (p={:0.2f})".format(N,d,p,t,p_value)

fig.update_layout(title=title,yaxis_title='Count',xaxis_title='value',font_size=16)

fig.show()


**Playing around with the 5 input parameters at the top of the cell, effect size tells us how "far apart" the mean of the two distributions is scaled by the standard deviations. A larger sample size is always preferred because it allows us to detect a smaller difference with the same confidence.**

**Lets look at binomial distributions since we often want to measure flags or alarms.**

In [118]:
# same but binomials
x1_mean = 0.65
x2_mean = 0.75
N = 500

df = pd.DataFrame(
    {'x1':np.random.binomial(n=1,p=x1_mean,size=N)
     ,'x2':np.random.binomial(n=1,p=x2_mean,size=N)
    })

fig = go.Figure()
fig.add_trace(go.Histogram(x=df['x1'],name='x1 p={:0.2f}'.format(df['x1'].mean())))
fig.add_vline(df['x1'].mean())
fig.add_trace(go.Histogram(x=df['x2'],name='x2 p={:0.2f}'.format(df['x2'].mean())))
fig.add_vline(df['x2'].mean())

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

# effect size in title
d = cohen_d(df['x1'],df['x2'])
p = power_analysis.solve_power(effect_size=d,power=0.8,alpha=0.05)
t,p_value = stats.ttest_ind(df['x1'].values,df['x2'].values)
title = "Binomial Distributions N={:0.0f} Effect Size = {:0.1f} (Power @ N={:0.1f}) t={:0.1f} (p={:0.2f})".format(N,d,p,t,p_value)

fig.update_layout(title=title,yaxis_title='Count',xaxis_title='value',font_size=16)
fig.show()


**Interesting, if the two probabilities are similar, there needs to be a large N to detect a difference.**